In [7]:
import sys
import math
import matplotlib.pyplot as plt
import numpy as np
import glob
import os

from numpy import asarray
from PIL import Image
from skimage.draw import ellipse
from skimage.measure import label, regionprops, find_contours, approximate_polygon
from skimage.transform import rotate
from sklearn.neighbors import KDTree
from scipy.spatial import distance

import hdbscan

### Uses numpy, skimage, pillow

In [2]:
class cell_poly:
    #centroid as a 1x2 tuple (x,y)
    #polygon as a list of 1x2 tuples (x,y)
    def __init__(self, centroid, polygon):
        self.centroid = centroid
        self.polygon = polygon

In [3]:
def cell_poly_properties(img):
    #takes in an array of a binary mask
    #finds all cell centroids and polygonal borders in a given mask
    #returns a cell array of user defined objects
    
    
    #img = asarray(Image.open(image_path))
    img = label(img)
    regions = regionprops(img)
    
    # SUGGESTED
    boundary = [approximate_polygon (contour, tolerance=1) for contour in find_contours (img, 0)]
    '''
    boundary = []
    for i,contour in enumerate(find_contours(img, 0)):
        boundary[i] = approximate_polygon(contour, tolerance = 1)
    '''
    
    # SUGGESTED
    cell_poly_list = [cell_poly (regions[i].centroid, boundary[i]) for i in range (0, len(boundary))]
    '''
    cell_poly_list = []
    for i,contour in enumerate(boundary):
        cell_poly_list[i] = cell_poly(regions[i].centroid, contour)
    return cell_poly_list
    '''

In [12]:
def make_adjacencies(clist, mindist):
    #takes a list of n cell_poly objects
    #takes a minimum distance defined as adjacent
    #returns an adjacency list of length n
    
    centroid_list = []
    adjacency_list = []
    wide_list = []
    
    centroid_list = [cell.centroid for cell in clist]
    '''
    for i,cell in enumerate(clist):
        centroid_list[i] = cell.centroid
    '''
    
    tree = KDTree(centroid_list, leafsize=10)
    
    wide_list = [tree.query_radius(center, mindist*10) for center in centroid_list]
    
    for i,small_list in enumerate(wide_list):
        adjacency_list.append (list())
        for index in small_list:
            # if we've already explored this pair then skip (as cdist _should_ be symmetrical)
            if index <= i:
                continue
            d = distance.cdist(clist[i].polygon,clist[index].polygon,'euclidean')
            if min(d) < mindist:
                adjacency_list[i].append (index)
    
    # TODO: correct all singly-matched pairs (which should be all of them)
    return adjacency_list

In [18]:
def HDB_cluster(centroids_list, min_size):
    clusterer = hdbscan.HDBSCAN(min_cluster_size=min_size)
    cluster_labels = clusterer.fit_predict(centroids_list)
    hierarchy = clusterer.cluster_hierarchy_
    alt_labels = hierarchy.get_clusters(0.100, 5)
    #hierarchy.plot()
    return (cluster_labels,hierarchy,alt_labels)
    
    #plot_clusters(data, hdbscan.HDBSCAN, (), {'min_cluster_size':15})

In [19]:
#driver

'''
images[]
for image_path in glob.glob(sys.argv[1] + '/*.png'): #filepath of folder
    #print(image_path)
    img = Image.open(image_path)
    images.append(asarray(img))
'''
#read in a list of polygons and bounding boxes
#or (temporary) a binary image
image_path = ''
img = asarray(Image.open(image_path))
clist = cell_poly_properties(img)
adj_list = make_adjacencies(clist, 10)


NameError: name 'image_path' is not defined

In [ ]:

fig, ax = plt.subplots()
ax.imshow(img[:,:,0], cmap=plt.cm.gray)


ax.plot((x0, x1), (y0, y1), '-r', linewidth=1)
plt.show()